In [77]:
:?

The following commands are available:
    :extension <Extension>    -  Enable a GHC extension.
    :extension No<Extension>  -  Disable a GHC extension.
    :type <expression>        -  Print expression type.
    :info <name>              -  Print all info for a name.
    :hoogle <query>           -  Search for a query on Hoogle.
    :doc <ident>              -  Get documentation for an identifier via Hoogle.
    :set -XFlag -Wall         -  Set an option (like ghci).
    :option <opt>             -  Set an option.
    :option no-<opt>          -  Unset an option.
    :?, :help                 -  Show this help text.
    :sprint <value>           -  Print a value without forcing evaluation.

Any prefix of the commands will also suffice, e.g. use :ty for :type.

Options:
  lint        – enable or disable linting.
  svg         – use svg output (cannot be resized).
  show-types  – show types of all bound names
  show-errors – display Show instance missing errors normally.
  pager       –

In [9]:
data T = A
       | B
       | C
       | D
       deriving Show


In [15]:
let values = [A, B, C, D]

In [20]:
:t print

print :: forall a. Show a => a -> IO ()

In [46]:
:t show

show :: forall a. Show a => a -> String

In [64]:
type T1 = Maybe Int

myprint :: T1 -> IO ()
myprint (Just t) = print ("Got int of " ++ show t)
myprint Nothing = print "Got nothing"

In [67]:
myprint (Just 10)

"Got int of 10"

In [68]:
myprint Nothing

"Got nothing"

In [71]:
:t myprint

myprint :: T1 -> IO ()

In [76]:
:info T1

type T1 :: *
type T1 = Maybe Int
  	-- Defined at <interactive>:1:1

---

In [78]:
:t fmap

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

In [80]:
double x = 2*x
:t double

double :: forall {a}. Num a => a -> a

In [110]:
data CountingType t = CountingType { val :: t, depth :: Int } deriving Show

instance Functor CountingType where
    fmap f (CountingType { val=v, depth=d }) = CountingType { val= f v, depth= d+1 }


In [120]:
let a = CountingType 1 0 
let b = CountingType 10 0
print a

CountingType {val = 1, depth = 0}

In [124]:
fdouble = fmap double
:t fdouble

fdouble :: forall {f :: * -> *} {b}. (Functor f, Num b) => f b -> f b

In [134]:
fdouble (fdouble (fmap (2 +) a))

CountingType {val = 12, depth = 3}

In [135]:
(*2) <$> a

CountingType {val = 2, depth = 1}

In [86]:
:t map

map :: forall a b. (a -> b) -> [a] -> [b]

---

In [165]:
data LogNum n = LogNum n String
    
logNum :: (Num n, Show n) => n -> LogNum n
logNum n = LogNum n (show n)

namedNum :: (Num n, Show n) => n -> String -> LogNum n
namedNum = LogNum

In [187]:
instance (Num n, Show n) => Num (LogNum n) where
    (+) (LogNum a la) (LogNum b lb) = LogNum (a+b) ("("++la++"+"++lb++")")
    (*) (LogNum a la) (LogNum b lb) = LogNum (a*b) ("("++la++"*"++lb++")")
    abs (LogNum a la) = LogNum (abs a) ("abs("++la++")")
    signum (LogNum a la) = LogNum (signum a) ("sign("++la++")")
    fromInteger a = LogNum (fromInteger a) (show a)
    negate (LogNum a la) = LogNum (negate a) ("(-"++la++") ")
    
instance (Num n, Show n) => Show (LogNum n) where
    show (LogNum a la) = "LogNum ("++(show a)++","++(show la)++")" 

Line 10: Redundant bracket
Found:
(show a) ++ "," ++ (show la) ++ ")"
Why not:
show a ++ "," ++ (show la) ++ ")"Line 10: Redundant bracket
Found:
(show la) ++ ")"
Why not:
show la ++ ")"

In [188]:
let a = logNum 5
(14-((a+10)*2))*3

LogNum (-48,"((14+(-((5+10)*2)) )*3)")

In [196]:
:t (<*>)

(<*>) :: forall (f :: * -> *) a b. Applicative f => f (a -> b) -> f a -> f b

In [251]:
:t (<$>)

(<$>) :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

In [255]:
:t pure (*) <$> [1,2,3] <*> [1,2]

pure (*) <$> [1,2,3] <*> [1,2] :: forall {a}. Num a => [a -> a]

In [244]:
(pure (5 *) <*> [1,2]) 
    == ((5 *) <$> [1,2])
(5*) <$> [10,20]

Line 1: Use <$>
Found:
pure (5 *) <*> [1, 2]
Why not:
(5 *) <$> [1, 2]

True

[50,100]

In [266]:
[(2*),(3*),(4*)] <*> [10,100]

[20,200,30,300,40,400]

In [268]:
(*) <$> [2,3,4] <*> [10,100]

[20,200,30,300,40,400]

In [277]:
pure (*) <*> [2,3,4] <*> [10,100]

Line 1: Use <$>
Found:
pure (*) <*> [2, 3, 4]
Why not:
((*) <$> [2, 3, 4])

[20,200,30,300,40,400]

In [282]:
[(*)] <*> [2,3,4] <*> [10,100]

[20,200,30,300,40,400]

In [285]:
a = [True, False]
tern True y z = y+z
tern False y z = y-z

tern <$> [True, False] <*> [10,20] <*> [1,2]

[11,12,21,22,9,8,19,18]

In [294]:
divn :: Int -> Int -> Maybe Int
divn n x
    | mod x n == 0 = Just (div x n)
    | otherwise    = Nothing
    
(divn 2) <$> [2,3,4,5,6,7]

Line 6: Redundant bracket
Found:
(divn 2) <$> [2, 3, 4, 5, 6, 7]
Why not:
divn 2 <$> [2, 3, 4, 5, 6, 7]

[Just 1,Nothing,Just 2,Nothing,Just 3,Nothing]

In [295]:
(divn 3) <$> [2,3,4,5,6,7]

Line 1: Redundant bracket
Found:
(divn 3) <$> [2, 3, 4, 5, 6, 7]
Why not:
divn 3 <$> [2, 3, 4, 5, 6, 7]

[Nothing,Just 1,Nothing,Nothing,Just 2,Nothing]

In [312]:
divn <$> [2,3] <*> [2,3,4,5,6,7] >>= pure (+10)

: 